# Lab 4. Retail Operations Manager

## Introduction

In this notebook we show you how to create multi-agent collaborator feature on Amazon Bedrock.

[Multi-agent Collaboration](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) is a Amazon Bedrock Agents native capability that enables a hierarchical collaboration between agents. You can now enable agent collaboration and associate secondary agents to a supervisor one. These secondary agents can be any existing agent within the same account, including agents that have collaboration themselves. This composable pattern allows you to build a chain of agents, as shown in the figure below.

![collaboration_hierarch](img/multi-agent_flow.png)

In this lab, we will create a supervisor agent that interacts with the subagents created in the previous lab.

The following represents the complete architecture of multi-agent collaborator.

![Architecture](img/energy_manager_agent.png)

This Lab will is divided into two notebooks:

- [4.1_retail_operations_manager.ipynb](4.1_retail_operations_manager.ipynb): Which contains Agent Setup
- [4.2_retail_operations_invocation.ipynb](4.2_retail_operations_invocation.ipynb): Which contains Agent Invocation

## Setup

Make sure that your boto3 version is the latest one.

If not, return no [notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) and run Setup block again.

In [ ]:
!pip freeze | grep boto3

Get your workshop ID / resource suffix. If not found, return to [notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) and run the Setup block again.

In [ ]:
import os

def get_workshop_id():
    workshop_id_file = '../.workshop_id'
    if os.path.exists(workshop_id_file):
        with open(workshop_id_file, 'r') as f:
            return f.read().strip()
    else:
        return None
    
workshop_id = get_workshop_id()
resource_suffix = f"{workshop_id}"

if workshop_id is None:
    print("No workshop ID found. Please run the Setup script in notebook 1.")
else:
    print("Your resource suffix is", resource_suffix)

### Creating Agent

On this section we're going to declare global variables that will be act as helpers during entire notebook and you will start to create your agent.

In [ ]:
import boto3

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

agent_foundation_model = [
    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0'
]

In [ ]:
retail_agent_name = f"retail-ops-{resource_suffix}"

retail_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{retail_agent_name}'

### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:

- `create_agent`: Create a new agent and respective IAM roles
- `associate_agents`: Associate sub-agents with multi-agent collaborator

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

Create the retail operations manager agent

In [ ]:
retail_agent = agents.create_agent(
    retail_agent_name,
    """
        Retail operations helper: Assists with sales forecasting, inventory, and store optimization.
    """,
    """
        You are a retail operations helper bot.
        You can retrieve sales forecasts and historical sales data for specific stores and help them optimize their inventory and operations.
        You can also retrieve inventory management information and inventory support tickets for specific stores and help them maintain proper stock levels.
        You can also get current information about store traffic patterns and can optimize staffing allocations.
        Resist the temptation to ask the user for input. Only do so after you have exhausted available actions. 
        Never ask the user for information that you already can retrieve yourself through available actions. 
    """,
    agent_foundation_model,
    agent_collaboration='SUPERVISOR_ROUTER'
)

retail_agent

Store environment variables to be used on next notebooks.

In [ ]:
retail_agent_id = retail_agent[0]
%store retail_agent_id
%store retail_agent_name

### Associate Collaborators

On this section, we're going to recover alias_id from previous agents (sub-agents) to add all of them inside the retail operations manager agent (which is multi-agent collaborator).

In [ ]:
%store -r
sales_agent_alias_arn, inventory_agent_alias_arn, store_ops_agent_alias_arn

In [ ]:
# If store fails, add Alias Arn here
#sales_agent_alias_arn = ''
#inventory_agent_alias_arn = ''
#store_ops_agent_alias_arn = ''

In [ ]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': sales_agent_alias_arn,
        'sub_agent_instruction': """Delegate sales data analysis and forecasting tasks to the Sales Forecasting Agent, ensuring adherence to its specific protocols and capabilities.""",
        'sub_agent_association_name': 'SalesForecastingAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': inventory_agent_alias_arn,
        'sub_agent_instruction': """Assign inventory management inquiries and issues to the Inventory Management Agent, respecting its scope and support ticket protocol.""",
        'sub_agent_association_name': 'InventoryManagementAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': store_ops_agent_alias_arn,
        'sub_agent_instruction': """Direct store operations optimization and staffing tasks to the Store Operations Manager Agent, leveraging its analytical capabilities.""",
        'sub_agent_association_name': 'StoreOperationsAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

Check if parameters are correclty formated

In [ ]:
sub_agents_list

Associate all sub-agents with this one (main one).

In [ ]:
retail_agent_alias_id, retail_agent_alias_arn = agents.associate_sub_agents(
    retail_agent_id, sub_agents_list
)

Check response return, to see all returned parameteres:

In [ ]:
retail_agent_alias_id, retail_agent_alias_arn

In [ ]:
%store retail_agent_alias_id
%store retail_agent_alias_arn

## Next Steps
Congratulations! We've now created a supervisor agent. Next let's invoke our agent